# Run this for once only

In [ ]:
!pip install requests pandas gspread oauth2client office365-runtime

# Now import

In [1]:
from files.constants import GET_ALL_FACILITIES_URL
from files.initialize import InitializeData
from files import get_all_facilities
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings("ignore")

# Initialize SpotHero reservation token for facility search
initialized_data = InitializeData()
all_facilities = get_all_facilities.getAllFacilities(
    initialized_data.auth['flex_auth'],
    GET_ALL_FACILITIES_URL
)




[token] reuse cached token.json (age=7.5m)
Getting all facilities
Successfully got all facilities


## Update inventory

In [2]:
from files.spothero_tool import run_update_for_facility_all_rules
import time

# --- knobs ---
ONLY_TIERED = True     # Dont change this to False
DRY_RUN = True         # Dont change this to False
DEBUG = True           # Dont change this to False
PAUSE_SEC = 0.2        

# 1) pick the facilities to process
df_fac = all_facilities.copy()
if ONLY_TIERED and "eventTieringEnabled" in df_fac.columns:
    df_fac = df_fac[df_fac["eventTieringEnabled"] == True]

# make sure we have the id column
if "id" not in df_fac.columns:
    raise ValueError("Expected column 'id' in all_facilities DataFrame.")

fids = df_fac["id"].dropna().astype(int).tolist()

print(f"About to process {len(fids)} facilities "
      f"({'tiered only' if ONLY_TIERED else 'all'}) | dry_run={DRY_RUN} debug={DEBUG}")

summaries = {}
failures = []

for i, fid in enumerate(fids, start=1):
    title = df_fac.loc[df_fac["id"] == fid, "title"].iloc[0] if "title" in df_fac.columns else ""
    addr  = df_fac.loc[df_fac["id"] == fid, "physicalAddress"].iloc[0] if "physicalAddress" in df_fac.columns else ""
    print(f"[{i}/{len(fids)}] facility {fid} — {title} | {addr}")

    try:
        summaries[fid] = run_update_for_facility_all_rules(
            auth_token=initialized_data.auth['flex_auth'],
            facility_id=int(fid),
            tz_name="America/Chicago",
            dry_run=DRY_RUN,
            debug=DEBUG
        )
    except Exception as e:
        failures.append((fid, str(e)))
        print(f"  ! failed: {e}")

    time.sleep(PAUSE_SEC)

print("\n=== run complete ===")
print(f"ok: {len(summaries)} | failed: {len(failures)}")
if failures:
    for fid, err in failures[:10]:
        print(f"  - {fid}: {err}")


example_fid = 72816
if example_fid in summaries:
    print("\n--- sample summary keys ---")
    print(summaries[example_fid].keys())


About to process 1161 facilities (tiered only) | dry_run=True debug=True
[1/1161] facility 41105 — 1 Coolidge Ave. - Spot #33 (ARCHIVED) | 1 Coolidge Avenue Boston MA 02114
  ! failed: No inventory rules for facility 41105 on 2025-12-03 (tz=America/Chicago)
[2/1161] facility 81099 — 1 Emmet Pl. - Left Spot Only | 1 Emmet Place Boston MA 02128
  ! failed: No inventory rules for facility 81099 on 2025-12-03 (tz=America/Chicago)
[3/1161] facility 60870 — 1 Snows Ct. NW - Parking Spot (ARCHIVED) | 1 Snows Court Northwest Washington DC 20037
  ! failed: No inventory rules for facility 60870 on 2025-12-03 (tz=America/Chicago)
[4/1161] facility 130104 — 1 Terrace St. (1430 Tremont St.) - Spot #11 or #14 Only | 1430 Tremont Street Boston MA 02120
  ! failed: No inventory rules for facility 130104 on 2025-12-03 (tz=America/Chicago)
[5/1161] facility 130666 — 10 Allen St. - Driveway Rear Spot Only | 10 Allen Street Cambridge MA 02140
  ! failed: No inventory rules for facility 130666 on 2025-12-

KeyboardInterrupt: 